### MLP with Tensorboard 4

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py

testing optimization loss function and gradient clipping

https://github.com/tensorflow/tensorflow/blob/c936c06a20b867eaa868e9f2c4fb53856391b18f/tensorflow/contrib/layers/python/layers/optimizers.py#L114

https://github.com/kapild/deeplearning/blob/8adb8452a8b16e15bdcec23153a78a3b1fa02f76/tensorflow/contrib/layers/python/layers/optimizers_test.py

will include test accuracy here

In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)=


# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf
import numpy as np
import input_data

In [3]:
%run infimnist.py

In [4]:
# Parameters
num_epochs = 5000
batch_size = 125

learning_rate = 0.001

In [5]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)


In [6]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

xt = tf.placeholder("float", [None, n_input])
yt = tf.placeholder("float", [None, n_classes])

In [7]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [9]:
# Construct model
mlp = multilayer_perceptron(x, weights, biases )
mlp_test = multilayer_perceptron(xt, weights, biases )

In [10]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(mlp, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

In [11]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

In [16]:
#with tf.name_scope("training accuracy"):
pred = tf.equal(tf.argmax(mlp, 1), tf.argmax(y, 1)) # Count correct predictions
train_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("training accuracy", train_acc_op)

<tf.Tensor 'ScalarSummary_3:0' shape=() dtype=string>

In [12]:
test_pred = tf.equal(tf.argmax(mlp_test, 1), tf.argmax(yt, 1)) # Count correct predictions
test_acc_op = tf.reduce_mean(tf.cast(test_pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("test accuracy", test_acc_op)

<tf.Tensor 'ScalarSummary_2:0' shape=() dtype=string>

In [13]:
infiminst = InfiMNIST()

In [14]:
!rm -rf ./logs/nn_logs_4

In [17]:
!ls logs

nn_logs_3


In [18]:
test_accuracies = []
train_accuracies = []
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs_4", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()
 
    tf.initialize_all_variables().run()

    for epoch in range(num_epochs):
        infiminst.next_epoch()
        mnist = input_data.read_data_sets(".", one_hot=True)
        
        trX, trY = mnist.train.images, mnist.train.labels
        teX, teY = mnist.test.images, mnist.test.labels
        
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size)):
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
                 
        summary, trn_acc, tst_acc = sess.run([merged, train_acc_op, test_acc_op], feed_dict={x: trX, y: trY, xt: teX, yt: teY})
        writer.add_summary(summary, epoch)  
        
        print(epoch, trn_acc, tst_acc)
        train_accuracies.append(trn_acc)
        test_accuracies.append(tst_acc)

        writer.flush()
        

(0, 0.94303638, 0.93989998)
(1, 0.95941818, 0.96069998)
(2, 0.9715091, 0.96710002)
(3, 0.97849089, 0.97640002)
(4, 0.98449093, 0.97939998)
(5, 0.98296362, 0.97649997)
(6, 0.986, 0.97979999)
(7, 0.98858184, 0.98210001)
(8, 0.98761821, 0.98220003)
(9, 0.98747271, 0.98329997)
(10, 0.98863637, 0.98189998)
(11, 0.99021816, 0.98290002)
(12, 0.99061817, 0.98519999)
(13, 0.98752725, 0.98070002)
(14, 0.99050909, 0.9842)
(15, 0.98930907, 0.9817)
(16, 0.98483634, 0.97750002)
(17, 0.98778182, 0.98259997)
(18, 0.98643637, 0.97850001)
(19, 0.99083638, 0.98409998)
(20, 0.98783636, 0.9817)
(21, 0.99269092, 0.98549998)
(22, 0.99403638, 0.98799998)
(23, 0.99150908, 0.9835)
(24, 0.99378181, 0.98549998)
(25, 0.99316365, 0.9867)
(26, 0.99174547, 0.98229998)
(27, 0.99385452, 0.98479998)
(28, 0.99379998, 0.98470002)
(29, 0.99434543, 0.98509997)
(30, 0.9939273, 0.98540002)
(31, 0.99332726, 0.98439997)
(32, 0.9950909, 0.98830003)
(33, 0.98825455, 0.98019999)
(34, 0.99281818, 0.98369998)
(35, 0.99487275, 0.9868

/Users/charlesmartin14/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


IOError: [Errno socket error] [Errno 8] nodename nor servname provided, or not known

In [ ]:
3)
(2206, 0.99989092, 0.98989999)
(2207, 0.99994546, 0.99119997)
(2208, 0.99980003, 0.99040002)
(2209, 0.99981821, 0.99019998)
(2210, 0.99989092, 0.98979998)
(2211, 0.99983639, 0.9903)
(2212, 0.99985456, 0.9896)
(2213, 0.99992728, 0.99059999)
(2214, 0.99992728, 0.99049997)
(2215, 0.99989092, 0.9896)
(2216, 0.99989092, 0.98890001)
(2217, 0.99996364, 0.9903)
(2218, 0.99992728, 0.98979998)
(2219, 0.99981821, 0.99000001)
(2220, 0.99983639, 0.98979998)
(2221, 0.99987274, 0.98940003)
(2222, 0.99989092, 0.9896)
(2223, 0.99989092, 0.99000001)
(2224, 0.99983639, 0.9892)
(2225, 0.99981821, 0.98909998)
(2226, 0.99994546, 0.98949999)
(2227, 0.99981821, 0.98860002)
(2228, 0.99987274, 0.9896)
(2229, 0.99985456, 0.99070001)
(2230, 0.99987274, 0.98909998)
(2231, 0.99992728, 0.99040002)
(2232, 0.99992728, 0.99119997)
(2233, 0.99992728, 0.9903)
(2234, 0.99985456, 0.99110001)
(2235, 0.99987274, 0.98989999)
(2236, 0.99976361, 0.9903)
(2237, 0.99976361, 0.99000001)
(2238, 0.99989092, 0.99070001)
(2239, 0.99969089, 0.98970002)
(2240, 0.99994546, 0.99040002)
(2241, 0.99994546, 0.99010003)
(2242, 0.99987274, 0.98989999)
(2243, 0.9999091, 0.99070001)
(2244, 0.9999091, 0.99089998)
(2245, 0.9999091, 0.98930001)
(2246, 0.99985456, 0.9903)
(2247, 0.9999091, 0.98989999)
(2248, 0.99981821, 0.98869997)
(2249, 0.99992728, 0.99019998)
(2250, 0.99987274, 0.98979998)
(2251, 0.99970907, 0.99019998)
(2252, 0.9999091, 0.99049997)
(2253, 0.99994546, 0.9903)
(2254, 0.99985456, 0.99040002)
(2255, 0.99996364, 0.99000001)
(2256, 0.99992728, 0.9903)
(2257, 0.99996364, 0.9903)
(2258, 0.99996364, 0.99070001)
(2259, 0.99994546, 0.99010003)
(2260, 0.9999091, 0.99089998)
(2261, 0.9999091, 0.991)
(2262, 0.9999091, 0.99059999)
(2263, 0.99994546, 0.99019998)
(2264, 0.99996364, 0.99089998)
(2265, 0.99996364, 0.99059999)
(2266, 0.99989092, 0.99019998)
(2267, 0.99989092, 0.99070001)
(2268, 0.99985456, 0.99019998)
(2269, 0.99996364, 0.99019998)
(2270, 0.99989092, 0.99059999)
(2271, 0.99978185, 0.98909998)
(2272, 0.9999091, 0.99019998)
(2273, 0.99994546, 0.98979998)
(2274, 0.99998182, 0.99010003)
(2275, 0.99994546, 0.9903)
(2276, 0.9999091, 0.9903)
(2277, 0.99981821, 0.99110001)
(2278, 0.99994546, 0.99010003)
(2279, 0.9999091, 0.99059999)
(2280, 0.99987274, 0.99150002)
(2281, 0.99987274, 0.9903)
(2282, 0.99996364, 0.9903)
(2283, 0.99983639, 0.99010003)
(2284, 0.99992728, 0.99040002)
(2285, 0.99974543, 0.98970002)
(2286, 0.99992728, 0.99000001)
(2287, 0.99989092, 0.98970002)
(2288, 0.99992728, 0.98879999)
(2289, 0.9999091, 0.99010003)
(2290, 0.99976361, 0.98979998)
(2291, 0.9999091, 0.9896)
(2292, 0.99989092, 0.9903)
(2293, 0.9999091, 0.98989999)
(2294, 0.99987274, 0.99110001)
(2295, 0.99987274, 0.99089998)
(2296, 0.99994546, 0.99150002)
(2297, 0.99994546, 0.99159998)
(2298, 0.99998182, 0.99110001)
(2299, 0.99994546, 0.98949999)
(2300, 0.99981821, 0.98940003)
(2301, 0.99992728, 0.98970002)
(2302, 0.99980003, 0.98970002)
(2303, 0.99994546, 0.99089998)
(2304, 0.99981821, 0.99089998)
(2305, 0.9999091, 0.99010003)
(2306, 0.99981821, 0.98989999)
(2307, 0.99985456, 0.9903)
(2308, 0.99987274, 0.98979998)
(2309, 0.99994546, 0.9892)
(2310, 0.99992728, 0.98989999)
(2311, 0.99980003, 0.98940003)
(2312, 0.99980003, 0.98940003)
(2313, 0.99989092, 0.98979998)
(2314, 0.99994546, 0.99089998)
(2315, 0.99998182, 0.98989999)
(2316, 0.99994546, 0.98949999)
(2317, 0.99992728, 0.98989999)
(2318, 0.99983639, 0.99000001)
(2319, 0.9999091, 0.98989999)
(2320, 0.99985456, 0.9903)
(2321, 0.99996364, 0.99080002)
(2322, 0.99976361, 0.99000001)
(2323, 0.99994546, 0.98940003)
(2324, 0.99987274, 0.9892)
(2325, 0.9999091, 0.98979998)
(2326, 0.99987274, 0.99000001)
(2327, 0.99985456, 0.9903)
(2328, 0.99976361, 0.9896)
(2329, 0.9999091, 0.98879999)
(2330, 0.9999091, 0.99000001)
(2331, 0.99992728, 0.99019998)
(2332, 0.99985456, 0.98869997)
(2333, 0.99987274, 0.98940003)
(2334, 0.99989092, 0.99019998)
(2335, 0.9999091, 0.99010003)
(2336, 0.99989092, 0.99070001)
(2337, 0.99983639, 0.98949999)
(2338, 0.99983639, 0.98970002)
(2339, 0.99996364, 0.99070001)
(2340, 0.99989092, 0.99059999)
(2341, 0.99996364, 0.98890001)
(2342, 0.99989092, 0.98909998)
(2343, 0.99983639, 0.99040002)
(2344, 0.99989092, 0.99040002)
(2345, 0.99985456, 0.9896)
(2346, 0.99985456, 0.99019998)
(2347, 0.99996364, 0.98970002)
(2348, 0.99985456, 0.99010003)
(2349, 0.99994546, 0.99059999)
(2350, 0.99994546, 0.991)
(2351, 0.99989092, 0.99070001)
(2352, 0.99994546, 0.98989999)
(2353, 0.9999091, 0.99049997)
(2354, 0.99992728, 0.99089998)
(2355, 0.99994546, 0.98970002)
(2356, 0.99994546, 0.99000001)
(2357, 0.99989092, 0.98940003)
(2358, 0.99994546, 0.9903)
(2359, 0.99985456, 0.99070001)
(2360, 0.9999091, 0.9903)
(2361, 0.99967271, 0.9903)
(2362, 0.99994546, 0.99119997)
(2363, 0.99994546, 0.99129999)
(2364, 0.9999091, 0.99000001)
(2365, 0.99996364, 0.98979998)
(2366, 0.99989092, 0.99019998)
(2367, 0.99994546, 0.9903)
(2368, 0.99994546, 0.99000001)
(2369, 0.9999091, 0.99000001)
(2370, 0.99987274, 0.9903)
(2371, 0.99992728, 0.99129999)
(2372, 0.99980003, 0.98909998)
(2373, 0.99985456, 0.99059999)
(2374, 0.99994546, 0.99019998)
(2375, 0.99985456, 0.99070001)
(2376, 0.99981821, 0.99119997)
(2377, 0.99994546, 0.98940003)
(2378, 0.99989092, 0.99059999)
(2379, 0.99989092, 0.99049997)
(2380, 0.9999091, 0.99040002)
(2381, 0.99987274, 0.98989999)
(2382, 0.9999091, 0.9903)
(2383, 0.99998182, 0.9903)
(2384, 0.99996364, 0.99000001)
(2385, 0.99983639, 0.98909998)
(2386, 0.99992728, 0.99010003)
(2387, 0.99998182, 0.99049997)
(2388, 0.99981821, 0.98940003)
(2389, 0.99978185, 0.98860002)
(2390, 0.9999091, 0.99019998)
(2391, 0.99996364, 0.98900002)
(2392, 0.99992728, 0.98989999)
(2393, 0.99996364, 0.9903)
(2394, 0.99987274, 0.99019998)
(2395, 0.99985456, 0.99000001)
(2396, 0.9999091, 0.9903)
(2397, 0.99985456, 0.99019998)
(2398, 0.99987274, 0.9903)
(2399, 0.99985456, 0.9892)
(2400, 0.99989092, 0.99010003)
(2401, 0.9999091, 0.99119997)
(2402, 0.9999091, 0.99040002)
(2403, 0.99987274, 0.99129999)
(2404, 0.9999091, 0.99059999)
(2405, 0.99983639, 0.991)
(2406, 0.99992728, 0.99180001)
(2407, 0.99992728, 0.9921)
(2408, 0.99989092, 0.99119997)
(2409, 0.99989092, 0.99169999)
(2410, 0.99989092, 0.99129999)
(2411, 0.99994546, 0.99110001)
(2412, 0.99987274, 0.991)
(2413, 0.9999091, 0.99129999)
(2414, 0.99996364, 0.99119997)
(2415, 0.99994546, 0.99059999)
(2416, 0.99985456, 0.99089998)
(2417, 0.99992728, 0.99059999)
(2418, 0.99972725, 0.99000001)
(2419, 0.99992728, 0.99080002)
(2420, 0.99989092, 0.99019998)
(2421, 0.9999091, 0.99000001)
(2422, 0.99980003, 0.98900002)
(2423, 0.99981821, 0.98979998)
(2424, 0.9999091, 0.98970002)
(2425, 0.99992728, 0.98979998)
(2426, 0.99987274, 0.98930001)
(2427, 0.99992728, 0.98989999)
(2428, 0.99978185, 0.98979998)
(2429, 0.99989092, 0.99040002)
(2430, 0.99994546, 0.99040002)
(2431, 0.99987274, 0.99019998)
(2432, 0.99987274, 0.9896)
(2433, 0.99987274, 0.9903)
(2434, 0.9999091, 0.98979998)
(2435, 0.99994546, 0.99019998)
(2436, 0.99992728, 0.99059999)
(2437, 0.99983639, 0.9903)
(2438, 0.99992728, 0.99019998)
(2439, 0.99989092, 0.98890001)
(2440, 0.99985456, 0.99019998)
(2441, 0.99992728, 0.99059999)
(2442, 0.99959999, 0.9885)
(2443, 0.99987274, 0.9903)
(2444, 0.99978185, 0.99000001)
(2445, 0.99992728, 0.99000001)
(2446, 0.99981821, 0.98930001)
(2447, 0.99994546, 0.99080002)
(2448, 0.9999091, 0.99080002)
(2449, 0.99983639, 0.99110001)
(2450, 0.99994546, 0.99070001)
(2451, 0.99980003, 0.98970002)
(2452, 0.99985456, 0.9903)
(2453, 0.99998182, 0.99019998)
(2454, 0.99992728, 0.99019998)
(2455, 0.99994546, 0.98979998)
(2456, 0.99996364, 0.99040002)
(2457, 0.99994546, 0.99059999)
(2458, 0.99989092, 0.9903)
(2459, 0.99980003, 0.99010003)
(2460, 0.99978185, 0.99010003)
(2461, 0.99989092, 0.99049997)
(2462, 0.9999091, 0.9903)
(2463, 0.99994546, 0.98970002)
(2464, 0.99985456, 0.99019998)
(2465, 0.99987274, 0.98949999)
(2466, 0.99992728, 0.98970002)
(2467, 0.99992728, 0.9903)
(2468, 0.99992728, 0.9896)
(2469, 0.99992728, 0.9903)
(2470, 0.99994546, 0.99000001)
(2471, 0.99989092, 0.98890001)
(2472, 0.99989092, 0.98940003)
(2473, 0.99989092, 0.98970002)
(2474, 0.99985456, 0.99010003)
(2475, 0.99992728, 0.9903)
(2476, 0.99994546, 0.98970002)
(2477, 0.99992728, 0.9903)
(2478, 0.9999091, 0.98900002)
(2479, 0.99989092, 0.99019998)
(2480, 0.9999091, 0.98940003)
(2481, 0.99994546, 0.99019998)
(2482, 0.99989092, 0.99010003)
(2483, 0.99987274, 0.99070001)
(2484, 0.99994546, 0.99070001)
(2485, 0.99980003, 0.9896)
(2486, 0.99981821, 0.99070001)
(2487, 0.99989092, 0.99089998)
(2488, 0.99994546, 0.98989999)
(2489, 0.99989092, 0.9903)
(2490, 0.99996364, 0.99110001)
(2491, 0.99994546, 0.99110001)
(2492, 0.99992728, 0.99129999)
(2493, 0.99994546, 0.99059999)
(2494, 0.9999091, 0.99049997)
(2495, 0.99994546, 0.99059999)
(2496, 0.99987274, 0.991)
(2497, 0.99994546, 0.99110001)
(2498, 0.99985456, 0.99010003)
(2499, 0.9999091, 0.98989999)
(2500, 0.99981821, 0.99080002)
(2501, 0.99996364, 0.99000001)
(2502, 0.99981821, 0.99010003)
(2503, 0.99998182, 0.99000001)
(2504, 0.99998182, 0.98949999)
(2505, 0.99983639, 0.99080002)
(2506, 0.99994546, 0.99070001)
(2507, 0.99992728, 0.99010003)
(2508, 0.99981821, 0.98970002)
(2509, 0.99994546, 0.98989999)
(2510, 0.99987274, 0.99010003)
(2511, 0.99992728, 0.99110001)
(2512, 0.99981821, 0.98979998)
(2513, 0.99987274, 0.99080002)
(2514, 0.99987274, 0.99040002)
(2515, 0.99992728, 0.99000001)
(2516, 0.99987274, 0.99010003)
(2517, 0.99996364, 0.98949999)
(2518, 0.99994546, 0.99049997)
(2519, 0.99994546, 0.99059999)
(2520, 0.99996364, 0.99089998)
(2521, 0.99983639, 0.99059999)
(2522, 0.99996364, 0.99049997)
(2523, 0.99992728, 0.99089998)
(2524, 0.99987274, 0.98930001)
(2525, 0.9999091, 0.98930001)
(2526, 0.99994546, 0.9896)
(2527, 0.99998182, 0.99070001)
(2528, 0.99992728, 0.9903)
(2529, 0.99996364, 0.99059999)
(2530, 0.99983639, 0.9903)
(2531, 0.99976361, 0.98799998)
(2532, 0.99996364, 0.9903)
(2533, 0.99994546, 0.9903)
(2534, 0.99994546, 0.99089998)
(2535, 0.9999091, 0.99070001)
(2536, 0.99989092, 0.99059999)
(2537, 0.99994546, 0.99019998)
(2538, 0.99992728, 0.99129999)
(2539, 0.99992728, 0.99010003)
(2540, 0.99996364, 0.99150002)
(2541, 0.99998182, 0.99070001)
(2542, 0.99994546, 0.99019998)
(2543, 0.99994546, 0.99040002)
(2544, 0.99980003, 0.99000001)
(2545, 0.99985456, 0.99040002)
(2546, 0.99996364, 0.99040002)
(2547, 0.99998182, 0.99040002)
(2548, 0.9999091, 0.99049997)
(2549, 0.99996364, 0.99010003)
(2550, 0.99989092, 0.99040002)
(2551, 0.99989092, 0.98979998)
(2552, 0.99996364, 0.99010003)
(2553, 0.99992728, 0.99080002)
(2554, 0.99992728, 0.99110001)
(2555, 0.99994546, 0.99150002)
(2556, 0.99996364, 0.99059999)
(2557, 0.99996364, 0.99040002)
(2558, 0.99980003, 0.99049997)
(2559, 0.99989092, 0.99059999)
(2560, 0.99996364, 0.99080002)
(2561, 0.99992728, 0.99010003)
(2562, 0.99992728, 0.99019998)
(2563, 0.99996364, 0.99049997)
(2564, 0.9999091, 0.99059999)
(2565, 0.99989092, 0.99019998)
(2566, 0.99994546, 0.99000001)
(2567, 0.99994546, 0.99000001)
(2568, 0.99998182, 0.98970002)
(2569, 0.99998182, 0.99019998)
(2570, 0.99992728, 0.9896)
(2571, 0.99992728, 0.99040002)
(2572, 0.99989092, 0.99040002)
(2573, 0.9999091, 0.9903)
(2574, 0.99994546, 0.98970002)
(2575, 0.99994546, 0.99049997)
(2576, 0.99989092, 0.98979998)
(2577, 0.99996364, 0.9903)
(2578, 0.99987274, 0.99080002)
(2579, 0.99989092, 0.9903)
(2580, 0.99996364, 0.99040002)
(2581, 0.99994546, 0.99059999)
(2582, 0.99996364, 0.99059999)
(2583, 0.9999091, 0.99049997)
(2584, 0.99994546, 0.98989999)
(2585, 0.99998182, 0.99049997)
(2586, 0.99992728, 0.98970002)
(2587, 0.9999091, 0.99049997)
(2588, 0.99992728, 0.99070001)
(2589, 0.9999091, 0.99040002)
(2590, 0.99989092, 0.99110001)
(2591, 0.99992728, 0.99070001)
(2592, 0.99996364, 0.99040002)
(2593, 0.99992728, 0.99000001)
(2594, 0.99989092, 0.9903)
(2595, 0.99994546, 0.99000001)
(2596, 0.99987274, 0.99059999)
(2597, 0.99994546, 0.99010003)
(2598, 0.99987274, 0.9903)
(2599, 0.99972725, 0.98940003)
(2600, 0.99989092, 0.99000001)
(2601, 0.99994546, 0.99019998)
(2602, 0.99987274, 0.98979998)
(2603, 0.99989092, 0.9903)
(2604, 0.99989092, 0.9896)
(2605, 0.99994546, 0.99049997)
(2606, 0.99985456, 0.9903)
(2607, 0.99983639, 0.9892)
(2608, 0.99989092, 0.99059999)
(2609, 0.99994546, 0.99049997)
(2610, 0.99998182, 0.98989999)
(2611, 0.99992728, 0.99000001)
(2612, 0.99994546, 0.99080002)
(2613, 0.99992728, 0.99119997)
(2614, 0.99985456, 0.99000001)
(2615, 0.99996364, 0.99019998)
(2616, 0.99992728, 0.99049997)
(2617, 0.99992728, 0.98979998)
(2618, 0.99994546, 0.99019998)
(2619, 0.99996364, 0.98949999)
(2620, 0.99996364, 0.99080002)
(2621, 0.9999091, 0.99010003)
(2622, 0.9999091, 0.99059999)
(2623, 0.99976361, 0.99070001)
(2624, 0.99992728, 0.99110001)
(2625, 0.99992728, 0.99059999)
(2626, 0.9999091, 0.99070001)
(2627, 0.99983639, 0.99049997)
(2628, 0.99994546, 0.99000001)
(2629, 0.99985456, 0.99070001)
(2630, 0.99985456, 0.9896)
(2631, 0.99994546, 0.98989999)
(2632, 0.99994546, 0.9903)
(2633, 0.99992728, 0.98989999)
(2634, 0.99994546, 0.99000001)
(2635, 0.99989092, 0.99040002)
(2636, 0.99996364, 0.99000001)
(2637, 0.9999091, 0.99040002)
(2638, 0.99996364, 0.99089998)
(2639, 0.99994546, 0.99059999)
(2640, 0.99998182, 0.99000001)
(2641, 0.99998182, 0.99059999)
(2642, 0.99992728, 0.99019998)
(2643, 0.99994546, 0.98940003)
(2644, 0.99998182, 0.9903)
(2645, 0.99998182, 0.99019998)
(2646, 0.99994546, 0.99010003)
(2647, 0.99998182, 0.99040002)
(2648, 0.99994546, 0.99000001)
(2649, 0.99987274, 0.9896)
(2650, 0.99998182, 0.98970002)
(2651, 0.99985456, 0.98979998)
(2652, 0.99998182, 0.99040002)
(2653, 0.99998182, 0.99019998)
(2654, 0.99992728, 0.99129999)
(2655, 0.99992728, 0.9903)
(2656, 0.99985456, 0.99049997)
(2657, 0.99987274, 0.99059999)
(2658, 0.9999091, 0.99080002)
(2659, 0.99980003, 0.99010003)
(2660, 0.99992728, 0.98989999)
(2661, 0.99994546, 0.9903)
(2662, 0.9999091, 0.99019998)
(2663, 0.99987274, 0.98979998)
(2664, 0.99998182, 0.991)
(2665, 0.99994546, 0.99070001)
(2666, 0.99987274, 0.991)
(2667, 0.99998182, 0.99110001)
(2668, 0.99972725, 0.98989999)
(2669, 0.99996364, 0.99080002)
(2670, 0.99981821, 0.99070001)
(2671, 0.99987274, 0.99089998)
(2672, 0.99989092, 0.99059999)
(2673, 0.99996364, 0.9914)
(2674, 0.99987274, 0.99040002)
(2675, 0.99996364, 0.98949999)
(2676, 0.99994546, 0.99000001)
(2677, 0.99994546, 0.99070001)
(2678, 0.99996364, 0.99070001)
(2679, 0.99994546, 0.98989999)
(2680, 0.99998182, 0.99089998)
(2681, 0.9999091, 0.9903)
(2682, 0.9999091, 0.99019998)
(2683, 0.99994546, 0.99089998)
(2684, 0.99983639, 0.99010003)
(2685, 0.99994546, 0.98970002)
(2686, 0.99994546, 0.99040002)
(2687, 0.99983639, 0.99000001)
(2688, 0.99961817, 0.98989999)
(2689, 0.9999091, 0.98949999)
(2690, 0.99992728, 0.99119997)
(2691, 0.99996364, 0.99010003)
(2692, 0.99994546, 0.98949999)
(2693, 0.9999091, 0.9896)
(2694, 0.99989092, 0.98860002)
(2695, 0.99989092, 0.99019998)
(2696, 0.99992728, 0.9892)
(2697, 0.9999091, 0.9896)
(2698, 0.99994546, 0.99019998)
(2699, 0.99978185, 0.98930001)
(2700, 0.9999091, 0.99010003)
(2701, 0.99994546, 0.991)
(2702, 0.99987274, 0.99010003)
(2703, 0.99994546, 0.99110001)
(2704, 0.99987274, 0.99000001)
(2705, 0.99992728, 0.9903)
(2706, 0.99978185, 0.99049997)
(2707, 0.99983639, 0.99040002)
(2708, 0.99992728, 0.99059999)
(2709, 0.99992728, 0.99110001)
(2710, 0.99994546, 0.99089998)
(2711, 0.99985456, 0.99080002)
(2712, 0.99992728, 0.99070001)
(2713, 0.99985456, 0.98970002)
(2714, 0.99992728, 0.99080002)
(2715, 0.99981821, 0.98909998)
(2716, 0.99981821, 0.99049997)
(2717, 0.99994546, 0.99059999)
(2718, 0.99996364, 0.99080002)
(2719, 0.99989092, 0.98979998)
(2720, 0.99994546, 0.98949999)
(2721, 0.99996364, 0.99019998)
(2722, 0.99981821, 0.9903)
(2723, 0.99994546, 0.9903)
(2724, 0.99996364, 0.99000001)
(2725, 0.99992728, 0.99019998)
(2726, 0.9999091, 0.99010003)
(2727, 0.99996364, 0.9903)
(2728, 0.99987274, 0.98949999)
(2729, 0.99992728, 0.9903)
(2730, 0.99981821, 0.9892)
(2731, 0.99980003, 0.99049997)
(2732, 0.99996364, 0.98940003)
(2733, 0.99994546, 0.98970002)
(2734, 0.99994546, 0.98949999)
(2735, 0.99998182, 0.98940003)
(2736, 0.99994546, 0.99019998)
(2737, 0.99996364, 0.98970002)
(2738, 0.99992728, 0.99070001)
(2739, 0.99996364, 0.98970002)
(2740, 0.99989092, 0.98949999)
(2741, 0.99998182, 0.98949999)
(2742, 0.99992728, 0.99010003)
(2743, 0.99996364, 0.98949999)
(2744, 0.9999091, 0.99000001)
(2745, 0.9999091, 0.99080002)
(2746, 0.99996364, 0.99049997)
(2747, 0.99992728, 0.98949999)
(2748, 0.99996364, 0.99000001)
(2749, 0.99996364, 0.98989999)
(2750, 0.99992728, 0.98989999)
(2751, 0.99981821, 0.98989999)
(2752, 0.99987274, 0.98869997)
(2753, 0.99989092, 0.98930001)
(2754, 0.99994546, 0.98989999)
(2755, 0.99980003, 0.98820001)
(2756, 0.99989092, 0.98989999)
(2757, 0.99987274, 0.98979998)
(2758, 0.99996364, 0.98970002)
(2759, 0.99987274, 0.99000001)
(2760, 0.99987274, 0.99019998)
(2761, 0.99998182, 0.98989999)
(2762, 0.99989092, 0.99010003)
(2763, 0.99992728, 0.99019998)
(2764, 0.99994546, 0.98930001)
(2765, 0.9999091, 0.98949999)
(2766, 0.99992728, 0.99010003)
(2767, 0.9999091, 0.98909998)
(2768, 0.99992728, 0.99059999)
(2769, 0.99998182, 0.9896)
(2770, 0.99992728, 0.99040002)
(2771, 0.99996364, 0.99040002)
(2772, 0.99996364, 0.98989999)
(2773, 0.99994546, 0.99070001)
(2774, 0.99989092, 0.99040002)
(2775, 0.99992728, 0.99019998)
(2776, 0.99998182, 0.99040002)
(2777, 0.99992728, 0.99010003)
(2778, 0.99996364, 0.99049997)
(2779, 0.99996364, 0.98989999)
(2780, 0.99989092, 0.99080002)
(2781, 0.9999091, 0.99049997)
(2782, 0.99987274, 0.99019998)
(2783, 0.99985456, 0.99089998)
(2784, 0.99996364, 0.9903)
(2785, 0.99989092, 0.99040002)
(2786, 0.99994546, 0.99080002)
(2787, 0.9999091, 0.99089998)
(2788, 0.99992728, 0.99119997)
(2789, 0.99985456, 0.99040002)
(2790, 0.99989092, 0.99089998)
(2791, 0.99974543, 0.99070001)
(2792, 0.99998182, 0.99080002)
(2793, 0.99994546, 0.99150002)
(2794, 0.99992728, 0.99080002)
(2795, 0.9999091, 0.99169999)
(2796, 0.99996364, 0.99040002)
(2797, 0.99989092, 0.99019998)
(2798, 0.99992728, 0.9896)
(2799, 0.99996364, 0.99019998)
(2800, 0.9999091, 0.99000001)
(2801, 0.99987274, 0.99000001)
(2802, 0.99994546, 0.99070001)
(2803, 0.99989092, 0.99080002)
(2804, 0.99992728, 0.99040002)
(2805, 0.99992728, 0.99080002)
(2806, 0.99989092, 0.99010003)
(2807, 0.99996364, 0.99089998)
(2808, 0.99994546, 0.99000001)
(2809, 0.99996364, 0.99059999)
(2810, 0.99992728, 0.9903)
(2811, 0.99987274, 0.99010003)
(2812, 0.99983639, 0.99019998)
(2813, 0.9999091, 0.9892)
(2814, 0.99987274, 0.98940003)
(2815, 0.9999091, 0.98979998)
(2816, 0.99996364, 0.98989999)
(2817, 0.99994546, 0.9896)
(2818, 0.99994546, 0.99070001)
(2819, 0.99985456, 0.99010003)
(2820, 0.99992728, 0.99040002)
(2821, 0.99994546, 0.98949999)
(2822, 0.99998182, 0.99049997)
(2823, 0.9999091, 0.99080002)
(2824, 0.99994546, 0.99010003)
(2825, 0.9999091, 0.98949999)
(2826, 0.99989092, 0.99010003)
(2827, 0.99994546, 0.99049997)
(2828, 0.99994546, 0.99010003)
(2829, 0.99994546, 0.99019998)
(2830, 0.99998182, 0.99049997)
(2831, 0.99996364, 0.98970002)
(2832, 0.99983639, 0.99010003)
(2833, 0.99987274, 0.99019998)
(2834, 0.99985456, 0.99010003)
(2835, 0.99994546, 0.98979998)
(2836, 0.99992728, 0.99019998)
(2837, 0.99981821, 0.98989999)
(2838, 0.99996364, 0.99019998)
(2839, 0.9999091, 0.99059999)
(2840, 0.99996364, 0.99070001)
(2841, 0.9999091, 0.98970002)
(2842, 0.99996364, 0.99010003)
(2843, 0.99992728, 0.99000001)
(2844, 0.9999091, 0.99089998)
(2845, 0.99996364, 0.99010003)
(2846, 0.99987274, 0.99080002)
(2847, 0.99983639, 0.99019998)
(2848, 0.99994546, 0.9914)
(2849, 0.9999091, 0.991)
(2850, 0.99994546, 0.99080002)
(2851, 0.99996364, 0.99070001)
(2852, 0.99996364, 0.99089998)
(2853, 0.9999091, 0.99000001)
(2854, 0.99996364, 0.99040002)
(2855, 0.99994546, 0.99080002)
(2856, 0.99992728, 0.98989999)
(2857, 0.99996364, 0.99010003)
(2858, 0.99989092, 0.98970002)
(2859, 0.9999091, 0.98979998)
(2860, 0.99994546, 0.99019998)
(2861, 0.99992728, 0.991)
(2862, 0.99989092, 0.99040002)
(2863, 0.99994546, 0.99080002)
(2864, 0.99996364, 0.99019998)
(2865, 0.99981821, 0.9896)
(2866, 0.99998182, 0.9896)
(2867, 0.99983639, 0.98979998)
(2868, 0.99998182, 0.99049997)
(2869, 0.99996364, 0.99040002)
(2870, 0.99994546, 0.99019998)
(2871, 0.99985456, 0.99019998)
(2872, 0.99994546, 0.99010003)
(2873, 0.99983639, 0.99040002)
(2874, 0.99989092, 0.99040002)
(2875, 0.99989092, 0.99010003)
(2876, 0.99983639, 0.99080002)
(2877, 0.99987274, 0.99080002)
(2878, 0.99981821, 0.99070001)
(2879, 0.99994546, 0.99059999)
(2880, 0.99983639, 0.99049997)
(2881, 0.99992728, 0.98989999)
(2882, 0.99987274, 0.99019998)
(2883, 0.9999091, 0.99019998)
(2884, 0.99992728, 0.98949999)
(2885, 0.99996364, 0.99010003)
(2886, 0.99996364, 0.99000001)
(2887, 0.99992728, 0.98989999)
(2888, 0.99996364, 0.99049997)
(2889, 0.9999091, 0.98839998)
(2890, 0.9999091, 0.9903)
(2891, 0.9999091, 0.99019998)
(2892, 0.99994546, 0.99000001)
(2893, 0.99998182, 0.99089998)
(2894, 0.9999091, 0.98949999)
(2895, 0.99994546, 0.99019998)
(2896, 0.99992728, 0.9892)
(2897, 0.99994546, 0.98970002)
(2898, 0.99992728, 0.99010003)
(2899, 0.99994546, 0.98989999)
(2900, 0.99994546, 0.99049997)
(2901, 0.99992728, 0.99059999)
(2902, 0.99994546, 0.98979998)
(2903, 0.99994546, 0.99019998)
(2904, 0.99996364, 0.99000001)
(2905, 0.99994546, 0.9896)
(2906, 0.99992728, 0.98979998)
(2907, 0.99989092, 0.98940003)
(2908, 0.99981821, 0.98970002)
(2909, 0.99994546, 0.98979998)
(2910, 0.99994546, 0.98930001)
(2911, 0.99994546, 0.98949999)
(2912, 0.99994546, 0.98989999)
(2913, 0.9999091, 0.99010003)
(2914, 0.9999091, 0.99019998)
(2915, 0.99992728, 0.99089998)
(2916, 0.99992728, 0.99040002)
(2917, 0.9999091, 0.99000001)
(2918, 0.99996364, 0.99049997)
(2919, 0.99987274, 0.9903)
(2920, 0.99994546, 0.98979998)
(2921, 0.99989092, 0.99010003)
(2922, 0.99981821, 0.98979998)
(2923, 0.99996364, 0.99049997)
(2924, 0.99996364, 0.9903)
(2925, 0.99970907, 0.9892)
(2926, 0.99985456, 0.98970002)
(2927, 0.99987274, 0.98940003)
(2928, 0.99981821, 0.98909998)
(2929, 0.99994546, 0.98930001)
(2930, 0.99998182, 0.98940003)
(2931, 0.99996364, 0.99000001)
(2932, 0.99980003, 0.98900002)
(2933, 0.99996364, 0.9892)
(2934, 0.99989092, 0.98879999)
(2935, 0.99985456, 0.98989999)
(2936, 0.99992728, 0.9892)
(2937, 0.99983639, 0.98989999)
(2938, 0.99998182, 0.98890001)
(2939, 0.99987274, 0.9896)
(2940, 0.99994546, 0.98940003)
(2941, 0.99983639, 0.9903)
(2942, 0.99987274, 0.98839998)
(2943, 0.99998182, 0.9896)
(2944, 0.99994546, 0.9903)
(2945, 0.99992728, 0.99000001)
(2946, 0.99985456, 0.98979998)
(2947, 0.99992728, 0.98940003)
(2948, 0.99996364, 0.99019998)
(2949, 0.99994546, 0.9896)
(2950, 0.99987274, 0.98900002)
(2951, 0.99987274, 0.98949999)
(2952, 0.99970907, 0.98970002)
(2953, 0.99981821, 0.9896)
(2954, 0.99985456, 0.99000001)
(2955, 0.99996364, 0.99040002)
(2956, 0.9999091, 0.98989999)
(2957, 0.99972725, 0.98900002)
(2958, 0.99996364, 0.99019998)
(2959, 0.99998182, 0.99019998)
(2960, 0.99992728, 0.99019998)
(2961, 0.99996364, 0.99070001)
(2962, 0.99994546, 0.99070001)
(2963, 0.99980003, 0.98989999)
(2964, 0.9999091, 0.99059999)
(2965, 0.99992728, 0.99059999)
(2966, 0.99992728, 0.99059999)
(2967, 0.9999091, 0.99049997)
(2968, 0.99994546, 0.99089998)
(2969, 0.99994546, 0.99110001)
(2970, 0.99989092, 0.99049997)
(2971, 0.99992728, 0.99089998)
(2972, 0.99992728, 0.99040002)
(2973, 0.99992728, 0.99040002)
(2974, 0.9999091, 0.98970002)
(2975, 0.9999091, 0.98989999)
(2976, 0.99994546, 0.98979998)
(2977, 0.99996364, 0.99010003)
(2978, 0.99998182, 0.98989999)
(2979, 0.99989092, 0.99019998)
(2980, 0.99981821, 0.9896)
(2981, 0.9999091, 0.99059999)
(2982, 0.99992728, 0.99059999)
(2983, 0.99998182, 0.99110001)
(2984, 0.99992728, 0.99040002)
(2985, 0.99981821, 0.99040002)
(2986, 0.99996364, 0.99059999)
(2987, 0.99996364, 0.99059999)
(2988, 0.99996364, 0.99000001)
(2989, 0.99994546, 0.99150002)
(2990, 0.99998182, 0.99059999)
(2991, 0.99985456, 0.99059999)
(2992, 0.9999091, 0.99080002)
(2993, 0.99987274, 0.99080002)
(2994, 0.99998182, 0.9914)
(2995, 0.99998182, 0.99129999)
(2996, 0.99994546, 0.99049997)
(2997, 0.99994546, 0.99080002)
(2998, 0.99992728, 0.99049997)
(2999, 0.99992728, 0.99049997)
(3000, 0.9999091, 0.99059999)
(3001, 0.99992728, 0.99000001)
(3002, 0.99992728, 0.99040002)
(3003, 0.99998182, 0.9903)
(3004, 0.99996364, 0.99049997)
(3005, 0.99994546, 0.9903)
(3006, 0.99992728, 0.99010003)
(3007, 0.99987274, 0.99019998)
(3008, 0.99992728, 0.98970002)
(3009, 0.99992728, 0.99059999)
(3010, 0.99981821, 0.99010003)
(3011, 0.99994546, 0.99150002)
(3012, 0.99989092, 0.99070001)
(3013, 0.99989092, 0.99040002)
(3014, 0.99992728, 0.99019998)
(3015, 0.99992728, 0.99089998)
(3016, 0.99992728, 0.9903)
(3017, 0.9999091, 0.99110001)
(3018, 0.99994546, 0.99119997)
(3019, 0.99996364, 0.9903)
(3020, 0.99989092, 0.99119997)
(3021, 0.99992728, 0.99049997)
(3022, 0.99996364, 0.99000001)
(3023, 0.99996364, 0.99059999)
(3024, 0.99974543, 0.98979998)
(3025, 0.99996364, 0.99000001)
(3026, 0.99983639, 0.99080002)
(3027, 0.99976361, 0.98900002)
(3028, 0.99992728, 0.99010003)
(3029, 0.99996364, 0.99040002)
(3030, 0.99992728, 0.99059999)
(3031, 0.99992728, 0.99070001)
(3032, 0.99996364, 0.99070001)
(3033, 0.99996364, 0.99080002)
(3034, 0.9999091, 0.9896)
(3035, 0.99994546, 0.99059999)
(3036, 0.99987274, 0.99059999)
(3037, 0.99996364, 0.991)
(3038, 0.99996364, 0.99119997)
(3039, 0.99994546, 0.99119997)
(3040, 0.99992728, 0.99080002)
(3041, 0.99992728, 0.99059999)
(3042, 0.99992728, 0.99150002)
(3043, 0.99994546, 0.99150002)
(3044, 0.9999091, 0.9896)
(3045, 0.99987274, 0.99080002)
(3046, 0.99987274, 0.99119997)
(3047, 0.99998182, 0.99110001)
(3048, 0.9999091, 0.9896)
(3049, 0.99992728, 0.99070001)
(3050, 0.99987274, 0.99000001)
(3051, 0.99996364, 0.991)
(3052, 0.99989092, 0.99070001)
(3053, 0.99992728, 0.99150002)
(3054, 0.99996364, 0.99159998)
(3055, 0.99994546, 0.99070001)
(3056, 0.99992728, 0.99000001)
(3057, 0.99996364, 0.99019998)
(3058, 0.99974543, 0.99110001)
(3059, 0.9999091, 0.99049997)
(3060, 0.99992728, 0.99070001)
(3061, 0.99994546, 0.99119997)
(3062, 0.99992728, 0.99049997)
(3063, 0.99985456, 0.98979998)
(3064, 0.99983639, 0.99019998)
(3065, 0.99994546, 0.99059999)
(3066, 0.99992728, 0.99059999)
(3067, 0.9999091, 0.99129999)
(3068, 0.99989092, 0.99119997)
(3069, 0.99994546, 0.99040002)
(3070, 0.99994546, 0.99040002)
(3071, 0.99987274, 0.9903)
(3072, 0.99992728, 0.991)
(3073, 0.99996364, 0.99169999)
(3074, 0.99996364, 0.99150002)
(3075, 0.99996364, 0.99070001)
(3076, 0.99994546, 0.99110001)
(3077, 0.99994546, 0.99080002)
(3078, 0.99981821, 0.99010003)
(3079, 0.99996364, 0.99129999)
(3080, 0.9999091, 0.991)
(3081, 0.99992728, 0.99080002)
(3082, 0.99987274, 0.99059999)
(3083, 0.99998182, 0.99049997)
(3084, 0.99987274, 0.99110001)
(3085, 0.99996364, 0.99159998)
(3086, 0.99992728, 0.99110001)
(3087, 0.9999091, 0.99089998)
(3088, 0.99996364, 0.99059999)
(3089, 0.99998182, 0.99049997)
(3090, 0.99992728, 0.99110001)
(3091, 0.99992728, 0.99119997)
(3092, 0.99998182, 0.99040002)
(3093, 0.99994546, 0.9903)
(3094, 0.99985456, 0.9903)
(3095, 0.99992728, 0.99059999)
(3096, 0.99989092, 0.98989999)
(3097, 0.99996364, 0.99040002)
(3098, 0.99998182, 0.99059999)
(3099, 0.99985456, 0.99089998)
(3100, 0.99992728, 0.99089998)
(3101, 0.99994546, 0.99110001)
(3102, 0.9999091, 0.99110001)
(3103, 0.99998182, 0.99019998)
(3104, 0.99994546, 0.991)
(3105, 0.99996364, 0.99080002)
(3106, 0.99983639, 0.99000001)
(3107, 0.99994546, 0.98930001)
(3108, 0.99985456, 0.99019998)
(3109, 0.99994546, 0.99019998)
(3110, 0.99989092, 0.99089998)
(3111, 0.99989092, 0.991)
(3112, 0.99989092, 0.98930001)
(3113, 0.9999091, 0.9903)
(3114, 0.99992728, 0.99000001)
(3115, 0.99992728, 0.99019998)
(3116, 0.99989092, 0.99049997)
(3117, 0.99994546, 0.99089998)
(3118, 0.99992728, 0.99010003)
(3119, 0.99998182, 0.99059999)
(3120, 0.99994546, 0.99000001)
(3121, 0.99992728, 0.9914)
(3122, 0.99976361, 0.99119997)
(3123, 0.99996364, 0.99110001)
(3124, 0.99992728, 0.9914)
(3125, 0.99996364, 0.98989999)
(3126, 0.99998182, 0.99059999)
(3127, 0.99989092, 0.98970002)
(3128, 0.9999091, 0.99019998)
(3129, 0.99996364, 0.98970002)
(3130, 0.99989092, 0.98979998)
(3131, 0.9999091, 0.99040002)
(3132, 0.99996364, 0.9903)
(3133, 0.99994546, 0.99010003)
(3134, 0.99996364, 0.99080002)
(3135, 0.99987274, 0.98989999)
(3136, 0.99987274, 0.99010003)
(3137, 0.9999091, 0.99010003)
(3138, 0.99992728, 0.99080002)
(3139, 0.99981821, 0.99010003)
(3140, 0.99983639, 0.99080002)
(3141, 0.99996364, 0.98949999)
(3142, 0.99992728, 0.99019998)
(3143, 0.99994546, 0.99089998)
(3144, 0.99996364, 0.99119997)
(3145, 0.99998182, 0.99070001)
(3146, 0.99985456, 0.99089998)
(3147, 0.99994546, 0.99010003)
(3148, 0.99994546, 0.991)
(3149, 0.99996364, 0.99089998)
(3150, 0.99998182, 0.99080002)
(3151, 0.99998182, 0.99119997)
(3152, 0.99992728, 0.99080002)
(3153, 0.99996364, 0.9914)
(3154, 0.99996364, 0.99119997)
(3155, 0.9999091, 0.99080002)
(3156, 0.99998182, 0.99119997)
(3157, 0.99985456, 0.99049997)
(3158, 0.99998182, 0.98979998)
(3159, 0.99996364, 0.98979998)
(3160, 0.99980003, 0.991)
(3161, 0.99996364, 0.99049997)
(3162, 0.99987274, 0.9903)
(3163, 0.99998182, 0.98989999)
(3164, 0.99994546, 0.99049997)
(3165, 0.99987274, 0.98989999)
(3166, 0.99998182, 0.98979998)
(3167, 0.99996364, 0.99070001)
(3168, 0.99989092, 0.99010003)
(3169, 0.9999091, 0.99010003)
(3170, 0.99996364, 0.9903)
(3171, 0.99980003, 0.99010003)
(3172, 0.9999091, 0.99049997)
(3173, 0.99996364, 0.98979998)
(3174, 0.99992728, 0.9896)
(3175, 0.99994546, 0.99049997)
(3176, 0.99996364, 0.99000001)
(3177, 0.99989092, 0.9896)
(3178, 0.99994546, 0.99010003)
(3179, 0.99980003, 0.99049997)
(3180, 0.9999091, 0.99000001)
(3181, 0.9999091, 0.9903)
(3182, 0.99994546, 0.98940003)
(3183, 0.99992728, 0.98989999)
(3184, 0.99998182, 0.99010003)
(3185, 0.99996364, 0.99010003)
(3186, 0.99989092, 0.99049997)
(3187, 0.99992728, 0.98930001)
(3188, 0.99985456, 0.98979998)
(3189, 0.99989092, 0.98949999)
(3190, 0.99967271, 0.98979998)
(3191, 0.99994546, 0.99010003)
(3192, 0.9999091, 0.99070001)
(3193, 0.9999091, 0.98930001)
(3194, 0.99987274, 0.99059999)
(3195, 0.99994546, 0.9914)
(3196, 0.99992728, 0.99089998)
(3197, 0.99994546, 0.9903)
(3198, 0.99996364, 0.99010003)
(3199, 0.99985456, 0.99019998)
(3200, 0.99992728, 0.99070001)
(3201, 0.9999091, 0.99089998)
(3202, 0.99996364, 0.99049997)
(3203, 0.99983639, 0.98940003)
(3204, 0.99989092, 0.99000001)
(3205, 0.99992728, 0.98940003)
(3206, 0.99992728, 0.98970002)
(3207, 0.99994546, 0.991)
(3208, 0.99992728, 0.98979998)
(3209, 0.99994546, 0.99019998)
(3210, 0.99992728, 0.99000001)
(3211, 0.9999091, 0.98989999)
(3212, 0.99998182, 0.9914)
(3213, 0.99996364, 0.99049997)
(3214, 0.99996364, 0.99010003)
(3215, 0.9999091, 0.99080002)
(3216, 0.99994546, 0.99019998)
(3217, 0.99998182, 0.99040002)
(3218, 0.99989092, 0.9903)
(3219, 0.9999091, 0.99010003)
(3220, 0.99994546, 0.99019998)
(3221, 0.99996364, 0.99040002)
(3222, 0.99989092, 0.99059999)
(3223, 0.9999091, 0.99040002)
(3224, 0.99992728, 0.99000001)
(3225, 0.99996364, 0.98979998)
(3226, 0.99989092, 0.9896)
(3227, 0.9999091, 0.99010003)
(3228, 0.99998182, 0.98989999)
(3229, 0.99992728, 0.99019998)
(3230, 0.99985456, 0.98900002)
(3231, 0.99994546, 0.98940003)
(3232, 0.99996364, 0.99080002)
(3233, 0.99998182, 0.99010003)
(3234, 0.99996364, 0.9896)
(3235, 0.99996364, 0.99040002)
(3236, 0.99989092, 0.99019998)
(3237, 0.99987274, 0.99049997)
(3238, 0.99996364, 0.99019998)
(3239, 0.99987274, 0.99040002)
(3240, 0.99987274, 0.99070001)
(3241, 0.99994546, 0.99110001)
(3242, 0.99989092, 0.99010003)
In [ ]:

​
In [ ]:

​
